In [1]:
# Import required libraries
import pickle
import copy
import pathlib
import dash
import math
import datetime as dt
import pandas as pd
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

from jupyter_dash import JupyterDash

In [17]:
from mapbox import Geocoder
mapbox_access_token = "pk.eyJ1IjoiamNyYW53ZWxsd2FyZCIsImEiOiJja2NkMW02aXcwYTl5MnFwbjdtdDB0M3oyIn0.zkIzPc4NSjLZvrY-DWrlZg"

geocoder = Geocoder(access_token=mapbox_access_token)

def geocode_address(address):
    """Geocode street address into lat/long."""
    response = geocoder.forward(address)
    coords = response.json()["features"][0]["center"]
    longitude = coords[0]
    latitude = coords[1]
    return dict(longitude=longitude, latitude=latitude)

In [2]:
sdmx_url = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.{}....?format=csv'

In [3]:
codes = [
    "EDU_SDG_STU_L2_MATH",
    "EDU_SDG_STU_L2_READING",
    "EDU_SDG_STU_L1_GLAST_MATH",
    "EDU_SDG_STU_L1_G2OR3_MATH",
    "EDU_SDG_STU_L1_GLAST_READING",
    "EDU_SDG_STU_L1_G2OR3_READING",
    "EDUNF_NERA_L2",
    "EDUNF_NERA_L1_PT",
    "EDU_SDG_GER_L01",
    "EDUNF_LR_L02",
    "EDUNF_ROFST_L2",
    "EDU_SDG_QUTP_L02",
    "EDU_SDG_QUTP_L1",
    "EDU_SDG_QUTP_L2",
    "EDU_SDG_QUTP_L3",
    "EDU_SDG_TRTP_L02",
    "EDU_SDG_TRTP_L1",
    "EDU_SDG_TRTP_L2",
    "EDU_SDG_TRTP_L3",
    "EDUNF_ROFST_L1", 
    "EDUNF_ROFST_L2", 
    "EDUNF_ROFST_L3", 
    "EDUNF_OFST_L1", 
    "EDUNF_OFST_L2", 
    "EDUNF_OFST_L3",
    "EDUNF_NERA_L1_GPIA",
    "EDUNF_NIR_L1_ENTRYAGE",
    "EDUNF_CR_L3",
    "EDUNF_NER_L02",
    "EDUNF_NERA_L1_GPIA",
    "EDUNF_NERA_L1_PT",
    "EDUNF_GER_L1",
    "EDUNF_GER_L2",
    "EDUNF_GER_L3_T",
    "EDUNF_NIR_L1_ENTRYAGE",
    "EDUNF_STU_L1_TOT",
    "EDUNF_STU_L2_TOT",
    "EDUNF_STU_L3_TOT"
]

df = pd.DataFrame()
inds = set(codes)
for ind in inds:
    sdmx = pd.read_csv(sdmx_url.format(ind))
    sdmx["CODE"] = ind
    df = df.append(sdmx)

In [5]:
countries = df["Geographic area"].unique()

In [6]:
df = df.merge(
    right=pd.DataFrame(
        [dict(country=country, **geocode_address(country)) for country in countries]
    ),
    left_on='Geographic area',
    right_on='country'
)

NameError: name 'geocode_address' is not defined

In [7]:
df[df["CODE"].isin(["EDUNF_NER_L02"]
            )].groupby(["CODE", "Indicator", "Geographic area", "TIME_PERIOD"]
            ).agg({"OBS_VALUE": "last"}
            ).reset_index()



,CODE,Indicator,Geographic area,TIME_PERIOD,OBS_VALUE
0,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Albania,1999,42.31086
1,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Albania,2000,42.77841
2,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Albania,2001,45.23442
3,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Albania,2003,48.33761
4,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Albania,2004,47.08262
...,...,...,...,...,...
411,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Uzbekistan,2014,19.34971
412,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Uzbekistan,2015,20.40027
413,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Uzbekistan,2016,20.39119
414,EDUNF_NER_L02,Enrolment in pre-primary (ISCED 02) education...,Uzbekistan,2017,22.91997


In [28]:
data

,CODE,Indicator,Geographic area,TIME_PERIOD,OBS_VALUE
0,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,1999,30461.00000
1,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2000,29656.00000
2,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2001,27237.00000
3,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2003,21197.00000
4,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2004,21996.00000
...,...,...,...,...,...
2419,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2013,13.82519
2420,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2014,17.76465
2421,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2015,15.83454
2422,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2016,15.69751


In [18]:
countries = pd.DataFrame([dict(name=country, **geocode_address(country)) for country in df["Geographic area"].unique()])

In [19]:
countries

,name,longitude,latitude
0,Albania,20.156691,40.641090
1,Armenia,44.660395,40.605656
2,Azerbaijan,47.720341,40.418260
3,Belarus,28.046788,53.539998
4,Georgia,-83.257256,32.330571
5,Croatia,16.679107,45.664952
6,Kyrgyzstan,74.555596,41.465054
7,Kazakhstan,67.301774,48.160088
8,Republic of Moldova,32.720174,39.823928
9,Serbia,20.795300,44.226737


In [23]:
data.merge(
    right=countries,
    left_on='Geographic area',
    right_on='name'
)

,CODE,Indicator,Geographic area,TIME_PERIOD,OBS_VALUE,name,longitude,latitude
0,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,1999,19659.000000,Albania,20.156691,40.641090
1,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2000,21103.000000,Albania,20.156691,40.641090
2,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2001,21787.500000,Albania,20.156691,40.641090
3,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2003,21092.000000,Albania,20.156691,40.641090
4,EDUNF_OFST_L1,Number of out of school children of primary sc...,Albania,2004,22114.500000,Albania,20.156691,40.641090
...,...,...,...,...,...,...,...,...
2419,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2013,14.460020,Uzbekistan,63.169372,41.750437
2420,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2014,17.612487,Uzbekistan,63.169372,41.750437
2421,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2015,16.168617,Uzbekistan,63.169372,41.750437
2422,EDUNF_ROFST_L3,Out of school children rate: Upper secondary e...,Uzbekistan,2016,16.463327,Uzbekistan,63.169372,41.750437


In [10]:
# Create controls
county_options = [
    {"label": str(country), "value": str(country)} for country in data['Geographic area'].unique()
]

In [11]:
county_options

[{'label': 'Albania', 'value': 'Albania'},
 {'label': 'Armenia', 'value': 'Armenia'},
 {'label': 'Azerbaijan', 'value': 'Azerbaijan'},
 {'label': 'Bosnia and Herzegovina', 'value': 'Bosnia and Herzegovina'},
 {'label': 'Bulgaria', 'value': 'Bulgaria'},
 {'label': 'Belarus', 'value': 'Belarus'},
 {'label': 'Czechia', 'value': 'Czechia'},
 {'label': 'Estonia', 'value': 'Estonia'},
 {'label': 'Georgia', 'value': 'Georgia'},
 {'label': 'Croatia', 'value': 'Croatia'},
 {'label': 'Hungary', 'value': 'Hungary'},
 {'label': 'Kyrgyzstan', 'value': 'Kyrgyzstan'},
 {'label': 'Kazakhstan', 'value': 'Kazakhstan'},
 {'label': 'Lithuania', 'value': 'Lithuania'},
 {'label': 'Latvia', 'value': 'Latvia'},
 {'label': 'Republic of Moldova', 'value': 'Republic of Moldova'},
 {'label': 'Montenegro', 'value': 'Montenegro'},
 {'label': 'North Macedonia', 'value': 'North Macedonia'},
 {'label': 'Poland', 'value': 'Poland'},
 {'label': 'Romania', 'value': 'Romania'},
 {'label': 'Serbia', 'value': 'Serbia'},
 {'

In [12]:
years =[i for i in range(2010, 2020)]

indicators = data['Indicator'].unique()

{index: str(year) for index,year in enumerate(years)}

{0: '2010',
 1: '2011',
 2: '2012',
 3: '2013',
 4: '2014',
 5: '2015',
 6: '2016',
 7: '2017',
 8: '2018',
 9: '2019'}

In [44]:

qutp = data[
    data["CODE"].isin([
            "EDU_SDG_QUTP_L02",
            "EDU_SDG_QUTP_L1",
            "EDU_SDG_QUTP_L2",
            "EDU_SDG_QUTP_L3",
        ])
    ].groupby(["TIME_PERIOD", "Geographic area"]
    ).agg({"OBS_VALUE": "mean"}
    ).reset_index()
qutp['Indicator'] = 'Proportion of teachers qualified according to national standards'

trtp = data[
        data["CODE"].isin([
                "EDU_SDG_TRTP_L02",
                "EDU_SDG_TRTP_L1",
                "EDU_SDG_TRTP_L2",
                "EDU_SDG_TRTP_L3",
            ])
    ].groupby(["TIME_PERIOD", "Geographic area"]
    ).agg({"OBS_VALUE": "mean"}
    ).reset_index()
trtp['Indicator'] = 'Proportion of teachers who have received at least the minimum organized teacher training'

df = qutp.append(trtp)
df.sort_values('TIME_PERIOD')
df.set_index('TIME_PERIOD')

# from mapbox import Geocoder

# mapbox_access_token = "pk.eyJ1IjoiamNyYW53ZWxsd2FyZCIsImEiOiJja2NkMW02aXcwYTl5MnFwbjdtdDB0M3oyIn0.zkIzPc4NSjLZvrY-DWrlZg"

# geocoder = Geocoder(access_token=mapbox_access_token)
# def geocode_address(address):
#     """Geocode street address into lat/long."""
#     response = geocoder.forward(address)
#     coords = response.json()['features'][0]['center']
#     # coords = coords.replace(']', '')
#     # coords = coords.replace('[', '')
#     coords

# data.apply(lambda row: pd.Series(geocode_address(row['Geographic area'])), axis=1)


,Geographic area,OBS_VALUE,Indicator
TIME_PERIOD,,,
1999,Kyrgyzstan,47.733180,Proportion of teachers qualified according to ...
2000,Kyrgyzstan,46.349310,Proportion of teachers qualified according to ...
2001,Kyrgyzstan,48.432090,Proportion of teachers qualified according to ...
2002,Kyrgyzstan,49.257690,Proportion of teachers qualified according to ...
2012,Uzbekistan,96.968067,Proportion of teachers qualified according to ...
...,...,...,...
2018,Kazakhstan,100.000000,Proportion of teachers who have received at le...
2018,Republic of Moldova,97.356703,Proportion of teachers who have received at le...
2018,Ukraine,87.300540,Proportion of teachers who have received at le...
